In [1]:
# convert mask images to label studio format

In [2]:
#!pip install -U label-studio-converter

In [3]:
from label_studio_converter import brush

No module named 'lxml'


In [4]:
from typing import Union, List, Callable
import numpy as np
import os
from pathlib import Path
import git
import json

In [5]:
def download_data(data_dir):
    git.Git(data_dir).clone("https://github.com/uwm-bigdata/wound-segmentation")

In [6]:
def get_data_dir() -> Path:
    data_dir = Path("../data")

    if not os.path.exists(data_dir):
        os.makedirs(data_dir, exist_ok=True)
    
    if "wound-segmentation" not in os.listdir(data_dir):
        download_data()
        
    dfu_dataset_path = data_dir / "wound-segmentation" / "data" / "Foot Ulcer Segmentation Challenge"
    return dfu_dataset_path

In [7]:
# local_dir = os.path.abspath(os.path.expanduser("~/dev/foothold/labelling/data"))
dfu_dataset_path = get_data_dir()

In [8]:
dfu_dataset_path

PosixPath('../data/wound-segmentation/data/Foot Ulcer Segmentation Challenge')

In [9]:
web_path = "https://raw.githubusercontent.com/uwm-bigdata/wound-segmentation/master/data/Foot%20Ulcer%20Segmentation%20Challenge"

In [10]:
def image_to_annotated_dict(image_path, mask_path):
    image_path = os.path.abspath(image_path)
    annotations = []
    if mask_path:
        mask_path = os.path.abspath(mask_path)
        annotations_dict = brush.image2annotation(
            mask_path,
            "Wound",
            "tag",
            "image",
            ground_truth=True,
            model_version=None,
            score=None
        )
        annotations.append(annotations_dict)
    task = {
        'data': {'image': str(image_path).replace(os.path.abspath(dfu_dataset_path), web_path)},
        'predictions': annotations
    }
    return task

In [13]:
image_path = Path(dfu_dataset_path) / "train" / "images" / "0011.png"
mask_path = Path(dfu_dataset_path) / "train" / "labels" / "0011.png"

In [14]:
task = image_to_annotated_dict(image_path, mask_path)

In [15]:
get_msk = lambda o: o.parent.parent / 'labels' / f'{o.stem}{o.suffix}'

In [49]:
tasks = []
path = dfu_dataset_path / "train" / "images"
for image in os.listdir(path):
    image_path = Path(path) / image
    mask_path = get_msk(image_path)
    task = image_to_annotated_dict(image_path, mask_path)
    tasks.append(task)

In [50]:
with open("./train.json", "w") as f:
    f.write(json.dumps(tasks))

In [53]:
tasks = []
path = dfu_dataset_path / "validation" / "images"
for image in os.listdir(path):
    image_path = Path(path) / image
    mask_path = get_msk(image_path)
    task = image_to_annotated_dict(image_path, mask_path)
    tasks.append(task)

In [54]:
with open("./validation.json", "w") as f:
    f.write(json.dumps(tasks))

In [58]:
tasks = []
path = dfu_dataset_path / "test" / "images"
for image in os.listdir(path):
    image_path = Path(path) / image
    mask_path = None
    task = image_to_annotated_dict(image_path, mask_path)
    tasks.append(task)

In [59]:
with open("./test.json", "w") as f:
    f.write(json.dumps(tasks))

In [16]:
# machine test labels

In [17]:
tasks = []
path = dfu_dataset_path / "test" / "images"
for image in os.listdir(path):
    image_path = Path(path) / image
    mask_path = get_msk(image_path)
    task = image_to_annotated_dict(image_path, mask_path)
    tasks.append(task)

In [18]:
with open("./test_machine.json", "w") as f:
    f.write(json.dumps(tasks))